In [1]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2
import joblib


2024-10-23 21:17:24.600584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 21:17:24.613002: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 21:17:24.616662: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 21:17:24.627478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 21:17:25.273639: W tensorflow/compiler/tf2

In [2]:
mp_holistic = mp.solutions.holistic #Holistic model 
mp_drawing = mp.solutions.drawing_utils #drawing utiliites
model_filename = 'LargerDataset.joblib'
model = joblib.load(model_filename)

In [4]:
def extract_keypoints(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # pose = pose[:22]
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ lh, rh])


In [5]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [13]:
IMAGE_Folder = 'example'
list_images = os.listdir(IMAGE_Folder)



In [14]:
predictions = {}
for image in list_images:
    image_path = os.path.join(IMAGE_Folder,image)
    img = cv2.imread(image_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        img, results = mediapipe_detection(img,holistic)
        keypoints = extract_keypoints(results)
        prediction = model.predict([keypoints])
        predictions[image_path] = prediction
        
    



I0000 00:00:1729693383.196068  309417 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1729693383.205175  317955 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: RENOIR (renoir, LLVM 15.0.7, DRM 3.57, 6.8.0-47-generic)
W0000 00:00:1729693383.256518  317938 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729693383.267500  317941 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729693383.268547  317947 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1729693383.268869  317937 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling suppo

In [15]:
predictions

{'example/44137383-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/4251ffdb-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['thank you'], dtype=object),
 'example/4251ffeb-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['take'], dtype=object),
 'example/472f92e8-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/472f92dc-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/4251ffdc-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['thank you'], dtype=object),
 'example/4251ffe0-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/42520000-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/44137362-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/42520004-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/4413738d-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'], dtype=object),
 'example/472f92d9-9140-11ef-99dc-bd7a4bfd03d8.jpg': array(['love'